In [1]:
import awkward as awk
import numpy as np
import uproot
from nanoframe import NanoFrame
import os
import particle
import pandas as pd
import uproot_methods

In [2]:
branches = ['BToKEE_charge',
 'BToKEE_chi2',
 'BToKEE_fit_cos2D',
 'BToKEE_fit_eta',
 'BToKEE_fit_mass',
 'BToKEE_fit_phi',
 'BToKEE_fit_pt',
 'BToKEE_kIdx',
 'BToKEE_l1Idx',
 'BToKEE_l2Idx',
 'BToKEE_l_xy',
 'BToKEE_l_xy_unc',
 'BToKEE_mll_fullfit',
 'BToKEE_mll_raw',
 'BToKEE_mass',
 'BToKEE_eta',
 'BToKEE_phi',
 'BToKEE_pt',
 'BToKEE_svprob',
 'Electron_isLowPt',
 'Electron_isPF',
 'Electron_eta',
 'Electron_mass',
 'Electron_mvaId',
 'Electron_phi',
 'Electron_pt',
 'Electron_vz',
 'HLT_Mu12_IP6_part0',
 'HLT_Mu12_IP6_part1',
 'HLT_Mu12_IP6_part2',
 'HLT_Mu12_IP6_part3',
 'HLT_Mu12_IP6_part4',
 'HLT_Mu7_IP4_part0',
 'HLT_Mu7_IP4_part1',
 'HLT_Mu7_IP4_part2',
 'HLT_Mu7_IP4_part3',
 'HLT_Mu7_IP4_part4',
 'HLT_Mu8_IP3_part0',
 'HLT_Mu8_IP3_part1',
 'HLT_Mu8_IP3_part2',
 'HLT_Mu8_IP3_part3',
 'HLT_Mu8_IP3_part4',
 'HLT_Mu8_IP5_part0',
 'HLT_Mu8_IP5_part1',
 'HLT_Mu8_IP5_part2',
 'HLT_Mu8_IP5_part3',
 'HLT_Mu8_IP5_part4',
 'HLT_Mu8_IP6_part0',
 'HLT_Mu8_IP6_part1',
 'HLT_Mu8_IP6_part2',
 'HLT_Mu8_IP6_part3',
 'HLT_Mu8_IP6_part4',
 'HLT_Mu9_IP4_part0',
 'HLT_Mu9_IP4_part1',
 'HLT_Mu9_IP4_part2',
 'HLT_Mu9_IP4_part3',
 'HLT_Mu9_IP4_part4',
 'HLT_Mu9_IP5_part0',
 'HLT_Mu9_IP5_part1',
 'HLT_Mu9_IP5_part2',
 'HLT_Mu9_IP5_part3',
 'HLT_Mu9_IP5_part4',
 'HLT_Mu9_IP6_part0',
 'HLT_Mu9_IP6_part1',
 'HLT_Mu9_IP6_part2',
 'HLT_Mu9_IP6_part3',
 'HLT_Mu9_IP6_part4',
 'Muon_pt',
 'Muon_mass',
 'Muon_eta',
 'Muon_phi',
 'Muon_pt',
 'Muon_vz',
 'Muon_isTriggering',
 'ProbeTracks_DCASig',
 'ProbeTracks_eta',
 'ProbeTracks_mass',
 'ProbeTracks_phi',
 'ProbeTracks_pt',
 'ProbeTracks_vz',
 'nBToKEE',
 'nElectron',
 'nMuon',
 'nProbeTracks']

In [3]:
nf = NanoFrame(
    '/eos/cms/store/cmst3/group/bpark/BParkingNANO_2019Sep10/ParkingBPH2/crab_data_Run2018D_part2/190910_082308/0001/BParkNANO_data_2019Sep10_1996.root',
    branches = branches
)

In [4]:
it = uproot.iterate(
    ['/eos/cms/store/cmst3/group/bpark/BParkingNANO_2019Sep10/ParkingBPH2/crab_data_Run2018D_part2/190910_082308/0001/BParkNANO_data_2019Sep10_1996.root'], 
    'Events', branches=branches, entrysteps=10000, namedecode='ascii'
)
nf = NanoFrame(it.__next__())

Load the needed collections, NanoFrame is just an empty shell until we call the collections

In [5]:
nf.array('Muon_pt')

<JaggedArray [[9.320822 1.0531727] [12.701347 9.697101 1.5778472] [13.613359 0.83395797] ... [9.099596] [10.143419] [10.332176]] at 0x7f3f9d948710>

In [6]:
muons = nf['Muon']

In [7]:
electrons = nf['Electron']

In [8]:
tracks = nf['ProbeTracks']

In [9]:
hlt = nf['HLT']

In [10]:
bcands = nf['BToKEE']

Attach the objects to the candidates

In [11]:
bcands['e1'] = electrons[bcands['l1Idx']]
bcands['e2'] = electrons[bcands['l2Idx']]
bcands['k'] = tracks[bcands['kIdx']]
bcands['p4fit'] = uproot_methods.TLorentzVectorArray.from_ptetaphi(
    bcands['fit_pt'], bcands['fit_eta'], bcands['fit_phi'], bcands['fit_mass']
)

Attach the trigger muon, identified as the closest in dz to the lead electron

In [12]:
muon_trg_mask = (muons.isTriggering == 1)
for path, pt_thr in [('Mu9_IP5', 9), ('Mu10p5_IP3p5', 10), ('Mu9_IP6', 9),
    ('Mu8p5_IP3p5', 8.5), ('Mu9_IP4', 9), ('Mu8_IP5', 8), ('Mu8_IP6', 8),
    ('Mu7_IP4', 7), ('Mu12_IP6', 12), ('Mu8_IP3', 8)]:
    if not any(path in i for i in hlt.columns): # the trigger is not here
        continue
    else:
        #merge all the parts and compute an or
        hlt_fired = np.hstack(
            [hlt[i].reshape((hlt[i].shape[0], 1)) for i in hlt.columns if path in i]
        ).any(axis = 1)
        muon_trg_mask = muon_trg_mask | (hlt_fired & (muons.p4.pt > pt_thr))

one_trg_muon = (muon_trg_mask.sum() != 0)

In [13]:
trig_mu = muons[muon_trg_mask][one_trg_muon]
bcands = bcands[one_trg_muon]

In [14]:
e1z, muz = bcands.e1.vz.cross(trig_mu.vz, nested = True).unzip()

In [15]:
closest_mu = np.abs(e1z - muz).argmin().flatten(axis = 1)

In [16]:
bcands['trg_mu'] = trig_mu[closest_mu]

Candidate selection, cut-based for the moment

In [17]:
b_selection = (bcands.k.p4.pt > 3.) & (bcands.k.DCASig > 2) & (bcands.p4fit.pt > 3) & \
    (bcands.svprob > 0.1) & (bcands.fit_cos2D > 0.999) & \
    ((bcands.l_xy / bcands.l_xy_unc) > 8)

/afs/cern.ch/work/m/mverzett/miniconda3/lib/python3.7/site-packages/awkward/array/jagged.py:1031: RuntimeWarning: divide by zero encountered in true_divide
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
b_pf = bcands.e1.isPF & bcands.e2.isPF
b_lpt = bcands.e1.isLowPt & bcands.e2.isLowPt & (bcands.e1.mvaId > 3.96) & (bcands.e1.mvaId > 3.96)

In [19]:
best_pf_cand = bcands[b_selection & b_pf].svprob.argmax()
bcands_pf = (bcands[b_selection & b_pf][best_pf_cand]).flatten()

In [20]:
best_lpt_cand = bcands[b_selection & b_lpt].svprob.argmax()
bcands_lpt = (bcands[b_selection & b_lpt][best_lpt_cand]).flatten()

In [21]:
dfs = {
    'pf' : pd.DataFrame(),
    'lpt' : pd.DataFrame(),
}

In [22]:
for name, tab, sel in [('pf', bcands_pf, b_selection & b_pf), ('lpt', bcands_lpt, b_selection & b_lpt)]:
    df = dfs[name]
    df['e1pt'] = tab.e1.p4.pt
    df['e2pt'] = tab.e2.p4.pt
    df['kpt'] = tab.k.p4.pt
    df['kDCA'] = tab.k.DCASig
    df['Bcharge'] = tab.charge
    df['Bpt'] = tab.p4fit.pt
    df['Beta'] = tab.p4fit.eta
    df['Bsvprob'] = tab.e2.p4.pt
    df['Bcos2D'] = tab.fit_cos2D
    df['Blxy_sig'] = (tab.l_xy / tab.l_xy_unc)
    df['Bmll'] = tab.mll_fullfit
    df['Bmll_raw'] = tab.mll_raw
    df['Bmass'] = tab.p4fit.mass
    df['nB'] = sel.sum()[sel.sum() != 0]

/afs/cern.ch/work/m/mverzett/miniconda3/lib/python3.7/site-packages/uproot_methods/classes/TLorentzVector.py:196: RuntimeWarning: invalid value encountered in sqrt
  return self._trymemo("mass", lambda self: self.awkward.numpy.sqrt(self.mag2))


In [ ]:
dfs['lpt'].to_hdf('out_data.h5', 'lpt', mode = 'a')
dfs['pf'].to_hdf('out_data.h5', 'pf', mode = 'a')